<h1> Create datasets B </h1>
<h2> 1. Import libraries and download datasets

In [2]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
train_D = pd.read_csv('data/train_df.csv')
val_D= pd.read_csv('data/val_df.csv')
test_D = pd.read_csv('data/test_df.csv')

<h2> 2. Filter dataset for relevant characters

In [28]:
imp_char = ["FRODO", "SAM", "GANDALF", "PIPPIN", "MERRY", "GOLLUM", "GIMLI", "THEODEN", "FARAMIR", "ARAGORN"]

def common_label_removal(data):
    mask = data["char"].isin(imp_char)
    data.loc[~ mask, "char"] = "Rest"
    mask2 = data['char'] == 'Rest'
    data = data[~mask2]
    return data

train_D = common_label_removal(train_D).reset_index(drop=True)
val_D = common_label_removal(val_D).reset_index(drop=True)
test_D = common_label_removal(test_D).reset_index(drop=True)

<h2> 3. Tokenize the dataset </h2>

In [32]:
def quote_list(X):
    quote_list = []
    for quote in range(len(X)):
        splitted_quote =  X['dialog'][quote].split()
        sequence_list = []
        for split in range(len(splitted_quote)):
            splitted_word = splitted_quote[split]

            word_list = str()
            i=0
            while i < (len(splitted_word)):
                # print(splitted_word[i])|
                if splitted_word[i].isalpha() == True:
                    word_list += splitted_word[i]
                i+=1
            sequence_list.append(word_list)
        quote_list.append(sequence_list)
    return quote_list

B1 = quote_list(train_D)
B2 = quote_list(val_D)
B3 = quote_list(test_D)

In [56]:
def maxlen(X):
    uni = []
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] not in uni:
                uni.append(X[i][j])
    return len(uni)
max_length = maxlen(B1)

<h2> 4. Tokenize, embed and pad the datasets

In [63]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(B1)
B1_seq = tokenizer.texts_to_sequences(B1)
B2_seq = tokenizer.texts_to_sequences(B2)
B3_seq = tokenizer.texts_to_sequences(B3)
maxlen = max([len(seq) for seq in B1_seq])

B1_padseq = pad_sequences(B1_seq, maxlen=max_length,padding='post')
B2_padseq = pad_sequences(B2_seq, maxlen=maxlen,padding='post')
B3_padseq = pad_sequences(B3_seq, maxlen=maxlen,padding='post')

In [64]:
train_B = pd.DataFrame(B1_padseq)
val_B = pd.DataFrame(B2_padseq)
test_B = pd.DataFrame(B3_padseq)
